In [ ]:
"""
Module: Slum Cluster Export Script

Script Functionality:
This script uses Google Earth Engine (GEE) to load a specific slum cluster dataset and exports it to Google Drive.
It retrieves the native resolution of the dataset, defines the region based on the dataset's geometry, 
and exports the image at its original resolution.

Input Parameters:
dataset: The slum cluster dataset, referenced by its Earth Engine asset ID ('users/EnvironmentalVariables/Slum/GHA_final_cluster').

Output:
An image file saved to Google Drive with the name 'GHA_final_cluster' at the dataset's native resolution.

Dependencies:
Earth Engine Python API : pip install earthengine-api (required for accessing Google Earth Engine)

Version Information:
Version: v1.0
Change Log: November 11, 2024
"""

import ee

# Initialize Earth Engine
ee.Initialize()

# 1. Load dataset (use your actual dataset ID)
dataset = ee.Image('users/EnvironmentalVariables/Slum/GHA_final_cluster')

# Get the native resolution of the image
native_resolution = dataset.projection().nominalScale().getInfo()

# Define the region using the geometry boundary of the dataset and ensure it is a polygon
region = dataset.geometry().bounds().getInfo()['coordinates']

# 2. Export the full image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=dataset,
    description='GHA_final_cluster',
    scale=native_resolution,  # Use native resolution
    region=region,  # Use the coordinates of the image geometry region
    maxPixels=1e13  # Ensure a sufficient maximum pixel count
)

# Start the task
task.start()
print("Export task started. Please check the task status in Google Drive.")


导出任务已启动，请检查Google Drive中的任务状态。


In [ ]:
"""


Module: Filtered Slum Cluster Export Script

Script Functionality:
This script uses the Google Earth Engine (GEE) API to filter and export specific slum cluster categories. 
It filters for two categories (6 and 7) in the provided dataset, creates a masked image showing only 
those categories, and then exports the result to Google Drive with the original resolution and projection.

Input Parameters:
dataset: The Earth Engine slum cluster dataset.

Output:
A masked image saved to Google Drive with the name 'Category6_7_Extraction', showing only categories 6 and 7.

Dependencies:
Earth Engine Python API : pip install earthengine-api (required for accessing Google Earth Engine)

Version Information:
Version: v1.0
Change Log: November 11, 2024
"""

import ee

# Initialize Earth Engine
ee.Initialize()

# 1. Load dataset (use actual dataset ID)
dataset = ee.Image('users/EnvironmentalVariables/Slum/GHA_final_cluster')

# 2. Filter categories (including category numbers 6 and 7)
category_6_or_7 = dataset.eq(6).Or(dataset.eq(7))  # Filter both category numbers 6 and 7

# 3. Create a masked image showing pixels of categories 6 and 7
category_6_7_image = dataset.updateMask(category_6_or_7)

# Get the native resolution of the image
native_resolution = dataset.projection().nominalScale().getInfo()

# Define the region using the geometry boundary of the dataset and ensure it is a polygon
region = dataset.geometry().bounds().getInfo()['coordinates']

# 4. Export the image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=category_6_7_image,
    description='Category6_7_Extraction',
    scale=native_resolution,  # Use native resolution
    region=region,  # Use the coordinates of the image geometry region
    maxPixels=1e13
)

# Start the task
task.start()
print("Export task started. Please check the task status in Google Drive.")


导出任务已启动，请检查 Google Drive 中的任务状态。


In [ ]:
"""

Module: Raster Proportion Calculation Script

Script Functionality:
This script calculates the proportion of specific target pixel values (default is 6 and 7) within a high-resolution raster, 
based on a specified template raster with a larger pixel size. It processes the high-resolution raster in chunks, calculates 
the percentage of target values in each section, and saves the result to an output raster file, matching the resolution 
and projection of the input high-resolution raster.


Input Parameters:
input_tif: Path to the input high-resolution raster TIF file.
template_tif: Path to the template raster file with a larger pixel size.
output_tif: Path to the output TIF file where the percentage results will be saved.
target_values: Tuple of target pixel values to calculate the percentage for, default is (6, 7).
chunk_size: Size of chunks to process in memory to avoid memory overflow issues.

Dependencies:
GDAL : pip install gdal 
NumPy : pip install numpy 

Version Information:
Version: v1.0
Change Log: November 11, 2024
"""

import os
import numpy as np
from osgeo import gdal

def calculate_value_percentage(input_tif, template_tif, output_tif, target_values=(6, 7), chunk_size=1024):
    """
    Calculate the percentage of areas with pixel values equal to target values (default is 6 and 7)
    based on 100-meter resolution, and write the result back to the original high-resolution raster.
    
    :param input_tif: Input high-resolution raster TIF file with 100-meter resolution
    :param template_tif: Template raster file with larger pixel size
    :param output_tif: Output TIF file for the percentage result
    :param target_values: Tuple of target pixel values to calculate percentage for, default is (6, 7)
    :param chunk_size: Size of chunks to process in memory to avoid MemoryError
    """
    # Open the input high-resolution raster file
    src_ds = gdal.Open(input_tif)
    if src_ds is None:
        print(f"Cannot open input TIF file: {input_tif}")
        return
    src_band = src_ds.GetRasterBand(1)
    
    # Open the template low-resolution raster file
    template_ds = gdal.Open(template_tif)
    if template_ds is None:
        print(f"Cannot open template TIF file: {template_tif}")
        return
    
    # Get geographic information of input and template
    src_transform = src_ds.GetGeoTransform()
    template_transform = template_ds.GetGeoTransform()

    # Set resolution to the actual pixel width of the input raster, 0.00089831528 degrees
    src_pixel_width = 0.00089831528  # Custom resolution provided
    template_pixel_width = template_transform[1]  # Pixel size of low-resolution template raster

    # Calculate the number of high-resolution pixels contained in each low-resolution template pixel
    ratio = int(round(template_pixel_width / src_pixel_width))

    # Get the size of the original high-resolution raster
    src_x_size = src_ds.RasterXSize
    src_y_size = src_ds.RasterYSize

    # Create output TIF file, maintaining the size of high-resolution raster
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(output_tif, src_x_size, src_y_size, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(src_transform)  # Use geographic information of original high-resolution raster
    out_ds.SetProjection(src_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)

    # Process the raster in chunks to avoid MemoryError
    for i in range(0, src_y_size, chunk_size):
        for j in range(0, src_x_size, chunk_size):
            # Define the chunk window for reading and processing
            x_chunk_size = min(chunk_size, src_x_size - j)
            y_chunk_size = min(chunk_size, src_y_size - i)

            # Read a chunk of the input high-resolution raster
            src_array = src_band.ReadAsArray(j, i, x_chunk_size, y_chunk_size)

            # Initialize output array for this chunk
            out_chunk_array = np.zeros((y_chunk_size, x_chunk_size), dtype=np.float32)

            # Process each sub-region within the chunk
            for y in range(0, y_chunk_size, ratio):
                for x in range(0, x_chunk_size, ratio):
                    # Get the area of the high-resolution raster within the template pixel range
                    sub_array = src_array[y:y+ratio, x:x+ratio]

                    # Count the number of pixels within the area with values equal to the target values (6 or 7)
                    target_count = np.sum(np.isin(sub_array, target_values))
                    total_count = sub_array.size

                    # Calculate proportion
                    proportion = target_count / total_count if total_count > 0 else 0

                    # Assign the proportion to all pixels in the original high-resolution raster
                    out_chunk_array[y:y+ratio, x:x+ratio] = proportion

            # Write the chunk to the output TIF file
            out_band.WriteArray(out_chunk_array, j, i)

    out_band.FlushCache()
    print(f"Percentage of areas with pixel values equal to {target_values} has been saved to: {output_tif}")

    # Close datasets
    src_ds = None
    template_ds = None
    out_ds = None


# Example usage
input_tif = r"D:\ghana\slum\Category6_7_Extraction.tif"  # Input high-resolution raster
template_tif = r"D:\ghana\0.005.tif"  # Template low-resolution raster
output_tif = r"D:\ghana\slum\slum_proportion_0.005.tif"  # Output TIF file path

calculate_value_percentage(input_tif, template_tif, output_tif)


Percentage of areas with pixel values equal to (6, 7) has been saved to: D:\ghana\slum\slum_proportion_0.005.tif


In [ ]:
"""


Module: Raster Resampling Script

Script Functionality:
This script resamples multiple input raster files based on the projection and resolution of a reference raster file. 
It reads raster files from an input folder, applies the projection and geo-transform of the reference raster, 
and saves each resampled file to an output folder.


Input Parameters:
referencefilepath: Path to the reference raster file (.tif) with the target resolution and projection.
inputfolderpath: Folder containing input raster files (.tif) to be resampled.
outputfolderpath: Folder to save the resampled output files.

Dependencies:
GDAL : pip install gdal 
NumPy : pip install numpy 

Version Information:
Version: v1.0
Change Log: November 11, 2024
"""

from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.005.tif'
inputfolderpath = r'D:\ghana\slum\0.005'
inputfilepaths = []
outputfolderpath = r'D:\ghana\slum\0.005'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, 'r' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

['D:\\ghana\\slum\\0.01\\rslum_proportion_0.005.tif', 'D:\\ghana\\slum\\0.01\\slum_proportion_0.005.tif']


In [ ]:
"""


Module: Raster Resolution Aggregation Script

Script Functionality:
This script processes all TIFF files in a specified input folder, resampling each from a high-resolution 
(0.01-degree) to a lower resolution (0.05-degree) by averaging. The resampled files are saved in a specified 
output folder with updated filenames.


Input Parameters:
input_folder: Path to the folder containing the input 0.01-degree resolution `.tif` files.
output_folder: Path to the folder where the resampled 0.05-degree `.tif` files will be saved.

Dependencies:
Rasterio : pip install rasterio (for raster processing and resampling)
NumPy : pip install numpy (for array manipulations)

Version Information:
Version: v1.0
Change Log: November 11, 2024
"""


import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\slum\0.01"  # Replace with your input folder path
output_folder = r"D:\ghana\slum\0.05"  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Mean aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).mean(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script

Processing file: D:\ghana\slum\0.01\slum_proportion_0.01.tif
File saved to: D:\ghana\slum\0.05\slum_proportion_0.05.tif
All files have been processed.
